In [9]:
!pip install -q transformers
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.6 MB/s eta 0:00:00


In [ ]:
# Trainer module based on pytorch , hence its manadatory , or we can also use keras to use trainer

In [10]:
from datasets import load_dataset

In [20]:
emotion_df = load_dataset("emotion")  #dataset of emotions , joy,anger etc. Multiclass calssification
emotion_df     # 16000 records for training data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [21]:
emotion_df['train'][0]  # first record from train df

{'text': 'i didnt feel humiliated', 'label': 0}

In [22]:
emotion_df['train'][15999]  # last record from train df  # text is data and label is column or Y

# Name of variables as "KEY" while records in it act as "values"

{'text': 'i know a lot but i feel so stupid because i can not portray it',
 'label': 0}

In [28]:
# converting this dictionary format with pandas df
train_df = emotion_df['train'].to_pandas()
train_df.head()

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3


In [30]:
features = emotion_df['train'].features  # MultiClass Classifiaction (6)classes
features # for datatypes of columns

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], id=None)}

In [33]:
# which label id , shows which class among 6 classes
features['label'].int2str(5)  #0,1,2,3,4,5

'surprise'

In [ ]:
# MAPPING OF EACH LABEL ID TO LABLE NAME  : useful when we do text classification using transformer , hence instead of giving output in int
# model return output in "Labels"

In [35]:
id2label = {idx:features['label'].int2str(idx) for idx in range(6)}
id2label

{0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}

In [37]:
label2id = {j:i for i,j in id2label.items() }
label2id

{'sadness': 0, 'joy': 1, 'love': 2, 'anger': 3, 'fear': 4, 'surprise': 5}

In [44]:
# Cheacking distribution of classes
train_df['label'].value_counts(normalize=True).sort_index() # Imbalaced dataset

# why not oversampling done while using transformers ?

0    0.291625
1    0.335125
2    0.081500
3    0.134937
4    0.121063
5    0.035750
Name: label, dtype: float64

In [ ]:
#Tokenize Data

In [45]:
from transformers import AutoTokenizer
model_ckpt = "microsoft/MiniLM-L12-H384-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [46]:
tokenizer(emotion_df['train']['text'][:1]) # encoding first record

{'input_ids': [[101, 1045, 2134, 2102, 2514, 26608, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}

In [47]:
emotion_df['train']['text'][0]

'i didnt feel humiliated'

In [49]:
# Tokenizer to all dataset (train,test, validate)
def tokenize_text(data):
  return tokenizer(data['text'],truncation= True,max_length=512)

In [52]:
emotion_df = emotion_df.map(tokenize_text,batched=True) #map is funtion of dataset library to aply finction to all data at a time
emotion_df

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [ ]:
# Dealing with imbalaced data
